In [2]:
%cd /home/jupyter/
!unzip streetview-kul.zip

/home/jupyter
Archive:  streetview-kul.zip
   creating: streetview-kul/
 extracting: streetview-kul/018b7188-b4a0-4c22-af1b-f029438f8ec3.jpg  
 extracting: streetview-kul/039cb1f9-efaa-404c-9bb0-644f307d476f.jpg  
 extracting: streetview-kul/02a81d56-3d97-48d0-b7a7-02a1663e1529.jpg  
 extracting: streetview-kul/022495aa-baaf-43fc-8028-d10cbcb68874.jpg  
 extracting: streetview-kul/0769fef7-2b38-4685-a8ee-2169d5dfa1ef.jpg  
 extracting: streetview-kul/035e2683-a561-430a-9bdd-4a79778174b8.jpg  
 extracting: streetview-kul/078ac399-5403-4edc-8c44-79c6ae1e2bf0.jpg  
 extracting: streetview-kul/08aff1fe-d384-46bc-9e42-f8c863d8667b.jpg  
 extracting: streetview-kul/0eacf583-3153-4ac4-a19e-c1cdae1809f7.jpg  
 extracting: streetview-kul/0add1472-d99c-4737-8dc3-cdfccb296dfd.jpg  
 extracting: streetview-kul/173b8fd4-4e7b-4fe2-b0e5-e4a6540300c3.jpg  
 extracting: streetview-kul/1552b251-d633-4a1d-8ddb-310c42e90695.jpg  
 extracting: streetview-kul/0a458c34-f24e-4b2d-a02e-4e65b7a437d6.jpg  
 extr

In [4]:
import json
%cd /home/jupyter/streetview-kul
#https://www.geeksforgeeks.org/convert-json-to-dictionary-in-python/
with open('_annotations.json') as json_file: 
    data = json.load(json_file) 
annotations = []
for item in data['annotations']:
    label = data['annotations'][item][0]['label']
    annotations.append((item, label))
annotations = dict(annotations)

/home/jupyter/streetview-kul


In [8]:
import shutil
base_path = '/home/jupyter/streetview-kul/'
for i in annotations:
    shutil.move(base_path + i, base_path + annotations[i] +"/" + i)

data augmentation

In [15]:
import tensorflow as tf

IMAGE_SIZE=(224,224)
data_dir = '/home/jupyter/streetview-kul-data/data-large/'
datagen_kwargs = dict(rescale=1./255, dtype = 'float32')
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=500,
                   interpolation="bilinear")
#valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
 #   **datagen_kwargs)
#valid_generator = valid_datagen.flow_from_directory(
 #   data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

for label in ["Good", "Bad"]:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      horizontal_flip=True,
      width_shift_range=0.4,
      shear_range=0.4, zoom_range=0.4,
      brightness_range = [0.5, 1.0],
      **datagen_kwargs)
    train_generator = train_datagen.flow_from_directory(
    data_dir, shuffle=True, **dataflow_kwargs, save_to_dir="/home/jupyter/streetview-kul-data/augmented-data-large/"+label, save_format="jpg", classes=[label])
    batch = next(train_generator)

Found 476 images belonging to 1 classes.
Found 175 images belonging to 1 classes.


In [17]:
label="Bad"
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      horizontal_flip=True,
      width_shift_range=0.3,
      shear_range=0.3, zoom_range=0.3,
      brightness_range = [1.0, 1.5],
      **datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    data_dir, shuffle=True, **dataflow_kwargs, save_to_dir="/home/jupyter/streetview-kul-data/augmented-data-large/"+label, save_format="jpg", classes=[label])
batch = next(train_generator)

Found 175 images belonging to 1 classes.


In [6]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import cv2
import PIL
import os
data_dir="/home/jupyter/streetview-kul-data/data-large"
aug_data_dir="/home/jupyter/streetview-kul-data/augmented-data-large"
comp_dir="/home/jupyter/compressed-data"
for direct in [data_dir, aug_data_dir]:
    print(direct)
    for label in os.listdir(data_dir):
        for filename in os.listdir(os.path.sep.join([direct,label])):
            imagePath = os.path.sep.join([direct, label, filename])
            image = cv2.imread(imagePath)
            image = load_img(imagePath, target_size=(224, 224))
            if direct == data_dir:
                image.save(os.path.sep.join([comp_dir, "original", label, filename]), optimize=True, quality=30) 
            elif direct == aug_data_dir:
                image.save(os.path.sep.join([comp_dir, "augmented", label, filename]), optimize=True, quality=30) 

/home/jupyter/streetview-kul-data/data-large
/home/jupyter/streetview-kul-data/augmented-data-large


IsADirectoryError: [Errno 21] Is a directory: '/home/jupyter/streetview-kul-data/augmented-data-large/Bad/.ipynb_checkpoints'

In [18]:
label="Bad"
for filename in os.listdir(os.path.sep.join([aug_data_dir,label])):
    imagePath = os.path.sep.join([direct, label, filename])
    image = cv2.imread(imagePath)
    image = load_img(imagePath, target_size=(224, 224))
    image.save(os.path.sep.join([comp_dir, "augmented", label, filename]), optimize=True, quality=30) 

In [7]:
len(os.listdir('/home/jupyter/streetview-kul-data/data-large/Bad'))

175

In [8]:
len(os.listdir('/home/jupyter/compressed-data/original/Bad'))

175

In [16]:
len(os.listdir('/home/jupyter/streetview-kul-data/augmented-data-large/Bad'))

351

In [19]:
len(os.listdir('/home/jupyter/compressed-data/augmented/Bad'))

350

In [12]:
len(os.listdir('/home/jupyter/streetview-kul-data/augmented-data-large/Good'))

476

In [13]:
len(os.listdir('/home/jupyter/compressed-data/augmented/Good'))

476